In [108]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

In [109]:

import nltk
import gensim
        
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
stoplist = nltk.corpus.stopwords.words('english')

def reviews():            
    r_i = 0
    review_item_iterator = iter(parse("../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
    for it in range(100000): #range(1000000):
        if it%10000==0:
            print("it="+str(it))
        review_item = next(review_item_iterator)
        r_i = r_i + 1
        if r_i%10000==0:
            print(str(r_i))
            print(preprocess_sentence)
        if "Cell Phones" in review_item["categories"]:
            reviewText = review_item["reviewText"]
            s_i = 0  
            for sentence in sent_detector.tokenize(reviewText):
                s_i = s_i + 1

                #label = review_item["asin"] + "_" + review_item["reviewerID"] + "_" + str(s_i)
                preprocess_sentence = gensim.utils.simple_preprocess(sentence)
                preprocess_sentence = [word for word in preprocess_sentence if word not in stoplist]

                yield preprocess_sentence
                

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def nextTokenizedSentence():
    yield [token for token in tokens if token not in stop and token not in exclude]



In [110]:
def nextTaggedSentence():
    for tokenizedSentence in reviews():
        #print(tokenizedSentence)
        tagged = nltk.pos_tag(tokenizedSentence)
        yield tagged

In [111]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases



In [112]:
def nextNounPhrase( ):
    for taggedSentence in nextTaggedSentence():
        grammar = r"""
          NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
            """
        cp = nltk.RegexpParser(grammar)
        if taggedSentence:
            noun_phrase_tagged = cp.parse(taggedSentence)
            noun_phrases=[]
            nouns=traverse(noun_phrase_tagged,noun_phrases)
            #print(nouns)
            for noun in nouns:
                yield noun
 

import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    doc_complete = [review_doc["buyerTranslationFeedback"] 
                    for review_doc in my_collection.find() 
                    if "buyerTranslationFeedback" in review_doc.keys() ]
else:
    my_collection = mydb['amazon_product_reviews']
    doc_complete = [review_doc["reviewText"] 
                    for review_doc in my_collection.find({ "categories" : "Cell Phones" }) ]
print("count of reviews: "+ str(len(doc_complete)))



import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
review_source = "amazon" #aliexpress

if review_source == "aliexpress":
    my_collection = mydb['reviews']
    cursor = my_collection.find() 
    textFieldName = "buyerTranslationFeedback"
else:
    my_collection = mydb['amazon_product_reviews']
    cursor = my_collection.find({ "categories" : "Cell Phones" })
    textFieldName = "reviewText"


In [113]:
from nltk import FreqDist

fdist = FreqDist(nextNounPhrase()) 



it=0
10000
['bye', 'bye', 'tracfone', 'fun']
it=10000
20000
['also', 'need', 'provide', 'part', 'sn', 'emei', 'mobile', 'acct', 'last', 'ssn', 'thus', 'recommended', 'steer', 'clear', 'device', 'resale', 'value', 'nill']
it=20000
30000
[]
it=30000
40000
['perfect', 'want']
it=40000
50000
['knowing', 'know', 'would', 'still', 'buy', 'exact', 'phone', 'anything', 'happened', 'one', 'resist', 'shiny', 'pink', 'things']
it=50000
60000
['amazon', 'kind', 'enough', 'deduct', 'cost', 'free', 'super', 'saver', 'shipping', 'refund', 'well', 'making', 'pay', 'cost', 'shipping', 'worthless', 'phone', 'shaped', 'piece', 'plastic', 'back']
it=60000
70000
['good', 'good', 'good', 'best', 'best', 'best', 'price', 'use', 'going', 'travel', 'provider']
it=70000
80000
['great', 'cel', 'phone', 'price', 'common', 'commands', 'rugged', 'design', 'make', 'safe', 'felt']
it=80000
90000
[]
it=90000
100000
['sure', 'actually', 'use', 'emergency', 'case']


In [114]:
frequent_words_freq = fdist.most_common(1000)
print(frequent_words_freq )
#fdist.plot()

[('phone', 40182), ('use', 4412), ('camera', 4285), ('phones', 4158), ('battery', 3782), ('time', 3698), ('service', 2798), ('quality', 2702), ('screen', 2543), ('features', 2521), ('reception', 2218), ('problem', 2205), ('cell', 2016), ('call', 1992), ('life', 1953), ('work', 1830), ('motorola', 1770), ('people', 1764), ('thing', 1751), ('problems', 1740), ('calls', 1734), ('way', 1696), ('card', 1647), ('software', 1622), ('day', 1490), ('months', 1482), ('mp', 1462), ('voice', 1459), ('days', 1416), ('feature', 1366), ('year', 1331), ('button', 1311), ('get', 1269), ('memory', 1250), ('pictures', 1242), ('number', 1231), ('lot', 1211), ('volume', 1190), ('times', 1162), ('years', 1138), ('buttons', 1124), ('charge', 1122), ('music', 1078), ('ringtones', 1072), ('verizon', 1052), ('things', 1039), ('player', 1026), ('speaker', 996), ('device', 987), ('price', 985), ('everything', 978), ('minutes', 968), ('samsung', 962), ('something', 955), ('customer', 922), ('display', 921), ('size

In [115]:
import gensim
model = gensim.models.Word2Vec.load('../download/word2vec/myamazonmodel.word2vec')

In [116]:
#frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words_freq)
import numpy as np
import itertools
#np_frequent_words = np.asarray([w for (w,c) in frequent_words_freq])
np_frequent_words  = []
[ np_frequent_words.extend([w]*int(c/100)) for (w,c) in frequent_words_freq]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [117]:
#print(np_frequent_words_lol[1:10])
#np_frequent_words = []
#map(np_frequent_words.extend, list(np_frequent_words_lol))
#print(np_frequent_words)
np_frequent_words_vector = np.asarray([model.wv[w] for w in np_frequent_words])
print(len(np_frequent_words_vector))

2601


In [118]:
import nltk
from nltk.cluster.kmeans import KMeansClusterer
nc = 10
data = np_frequent_words_vector

kclusterer = KMeansClusterer(nc, distance=nltk.cluster.util.cosine_distance, repeats=10,avoid_empty_clusters=True)
assigned_clusters = kclusterer.cluster(data, assign_clusters=True)
print('Means:', kclusterer.means())

Means: [array([ 1.51445711, -0.5430755 ,  3.0157814 , -1.68180799,  0.63127553,
       -3.20733833, -1.22747469,  0.04517423,  1.58403528,  1.6520381 ,
       -0.66313547,  0.26647025,  1.12175179, -2.19865346, -2.56030345,
        2.32973957,  3.13373113, -0.42323455, -2.50865459, -0.05362314,
        0.60883194,  1.14611518, -0.28045359,  0.81401646, -2.70091033,
        0.07994318, -0.00744976,  0.76255357, -2.4143312 , -2.90436864,
       -2.66441393, -0.52412784, -2.62783408,  1.22308242,  4.48455715,
       -2.94137955, -1.06831777, -1.4100641 , -1.77881682,  1.5392729 ,
       -5.37961006, -1.83747971,  2.53020716,  0.56502002, -3.68589783,
        1.45402181, -0.16841735, -1.171031  , -1.81544363, -1.08770144,
       -2.38298464,  1.14681184, -1.15323973,  1.49410605, -0.31814212,
       -0.25020114, -1.78431082,  3.87958741, -2.63711762,  3.01316261,
       -2.11043096,  0.33860388, -1.66635501,  0.10769501, -2.01472664,
       -3.43329334, -1.09785032,  0.31336793, -1.1358500

In [119]:
print('#Means:', len(kclusterer.means()))
#print(assigned_clusters)
np_assigned_clusters = np.asarray(assigned_clusters)

#Means: 10


In [120]:
# get nearest words for centroids
#print(kclusterer.means()[0])
[print(model.wv.most_similar([m])) for m in kclusterer.means()]

[('days', 0.9069166779518127), ('weeks', 0.8679072260856628), ('months', 0.8648396730422974), ('week', 0.859013020992279), ('month', 0.8467245101928711), ('hours', 0.8400271534919739), ('minutes', 0.8057142496109009), ('mins', 0.7944138050079346), ('years', 0.7936934232711792), ('year', 0.787887692451477)]
[('phone', 0.9969353079795837), ('cellphone', 0.7273889183998108), ('iphone', 0.7162175178527832), ('device', 0.6138522624969482), ('fone', 0.5366050004959106), ('droid', 0.5173928737640381), ('property', 0.5155690908432007), ('evo', 0.5143734812736511), ('treo', 0.5043409466743469), ('droidx', 0.4985789358615875)]
[('color', 0.7299684882164001), ('appearance', 0.7070344686508179), ('texture', 0.7061024308204651), ('colour', 0.6797312498092651), ('design', 0.6786229610443115), ('pattern', 0.6711481213569641), ('shape', 0.6580385565757751), ('case', 0.6402664184570312), ('coloring', 0.627589225769043), ('shade', 0.6272152662277222)]
[('nokia', 0.7153540253639221), ('kyocera', 0.711591

[None, None, None, None, None, None, None, None, None, None]

In [121]:
import pandas as pd
#cluster_sentences_join = [" ".join(s_list) for s_list in cluster_sentences]
result = pd.DataFrame({ 'cluster' : assigned_clusters,   'feature' : np_frequent_words})
print(result.head())

   cluster feature
0        1   phone
1        1   phone
2        1   phone
3        1   phone
4        1   phone


In [122]:
writer = pd.ExcelWriter('clustered_features.xlsx')
result.to_excel(writer,'Sheet1')
writer.save()

In [123]:
features=  [ {"feature" : key} for key in frequent_words]
print(features)

[{'feature': 'phone'}, {'feature': 'battery'}, {'feature': 'screen'}, {'feature': 'time'}, {'feature': 'phones'}, {'feature': 'camera'}, {'feature': 'service'}, {'feature': 'card'}, {'feature': 'quality'}, {'feature': 'problem'}, {'feature': 'features'}, {'feature': 'cell'}, {'feature': 'iphone'}, {'feature': 'life'}, {'feature': 'price'}, {'feature': 've'}, {'feature': 'use'}, {'feature': 'product'}, {'feature': 'day'}, {'feature': 'calls'}, {'feature': 'thing'}, {'feature': 'keyboard'}, {'feature': 'way'}, {'feature': 'days'}, {'feature': 'problems'}, {'feature': 'call'}, {'feature': 'don'}, {'feature': 'amazon'}, {'feature': 'work'}, {'feature': 'data'}, {'feature': 'people'}, {'feature': 'apps'}, {'feature': 'months'}, {'feature': 'device'}, {'feature': 'minutes'}, {'feature': 'everything'}, {'feature': 'software'}, {'feature': 'internet'}, {'feature': 'lot'}, {'feature': 'year'}, {'feature': 'music'}, {'feature': 'plan'}, {'feature': 'years'}, {'feature': 'nokia'}, {'feature': 'mo

import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
try:
    mydb.features.drop()
finally:
    result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

my_collection.insert_many(features)

In [124]:
print(model.wv.most_similar("thing"))

[('downfall', 0.6178915500640869), ('drawback', 0.5932508707046509), ('downside', 0.5874196887016296), ('complaint', 0.5643346309661865), ('gripe', 0.553368091583252), ('quirk', 0.5493331551551819), ('disadvantage', 0.5491840839385986), ('downer', 0.5404207110404968), ('criticism', 0.529247522354126), ('shortcoming', 0.5244871973991394)]
